In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
data = keras.datasets.imdb

In [20]:
(train_data,train_labels),(test_data,test_labels) = data.load_data(num_words=88000)

In [21]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [22]:
word_index = keras.datasets.imdb.get_word_index()

In [23]:
word_index = {k:(v+3) for k,v in word_index.items()}

In [24]:
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3


In [25]:
reverse_word_index = dict(
    [(value, key) for (key,value) in word_index.items()]
)

In [26]:
def decode_review(text):
    return " ".join([reverse_word_index.get(i,"?") for i in text])

In [27]:
print(test_data[0])
print(decode_review(test_data[0]))

[1, 591, 202, 14, 31, 6, 717, 10, 10, 18142, 10698, 5, 4, 360, 7, 4, 177, 5760, 394, 354, 4, 123, 9, 1035, 1035, 1035, 10, 10, 13, 92, 124, 89, 488, 7944, 100, 28, 1668, 14, 31, 23, 27, 7479, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 15387, 38, 32, 25, 7944, 451, 202, 14, 6, 717]
<START> please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss


In [28]:
print(len(train_data),len(test_data))

25000 25000


In [29]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

In [30]:
print(len(test_data[0]),len(test_data[1]),len(test_data[2]),len(test_data[3]),len(test_data[4]),len(test_data[5]))

250 250 250 250 250 250


In [31]:
model = keras.Sequential()
model.add(keras.layers.Embedding(88000,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation="relu"))
model.add(keras.layers.Dense(1,activation="sigmoid"))#is review good or bad => 0 or 1

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [34]:
x_val = train_data[:10000] 
x_train = train_data[10000:]
y_val = train_labels[:10000] 
y_train = train_labels[10000:]

model.fit(x_train,y_train,epochs=40,batch_size=512, validation_data=(x_val,y_val), verbose=1) 
#Verbose is like, how detailed of an output is to be printed. Verbose = 2 will give just the accuracy and loss values
#after each epoch is trained. Verbose=0 will not print anything. Try changing the value of  verbose and see for yourself.

Epoch 1/40
30/30 [==============================] - 1s 34ms/step - loss: 0.6921 - accuracy: 0.5412 - val_loss: 0.6905 - val_accuracy: 0.5101
Epoch 2/40
30/30 [==============================] - 1s 30ms/step - loss: 0.6870 - accuracy: 0.6501 - val_loss: 0.6837 - val_accuracy: 0.6984
Epoch 3/40
30/30 [==============================] - 1s 35ms/step - loss: 0.6763 - accuracy: 0.6946 - val_loss: 0.6706 - val_accuracy: 0.7521
Epoch 4/40
30/30 [==============================] - 1s 29ms/step - loss: 0.6569 - accuracy: 0.7709 - val_loss: 0.6489 - val_accuracy: 0.7672
Epoch 5/40
30/30 [==============================] - 1s 29ms/step - loss: 0.6273 - accuracy: 0.8003 - val_loss: 0.6185 - val_accuracy: 0.7886
Epoch 6/40
30/30 [==============================] - 1s 32ms/step - loss: 0.5881 - accuracy: 0.8193 - val_loss: 0.5814 - val_accuracy: 0.8028
Epoch 7/40
30/30 [==============================] - 1s 33ms/step - loss: 0.5427 - accuracy: 0.8360 - val_loss: 0.5413 - val_accuracy: 0.8173
Epoch 8/40
30

In [35]:
results = model.evaluate(test_data,test_labels)

782/782 [==============================] - 1s 1ms/step - loss: 0.3286 - accuracy: 0.8717


In [36]:
print(results)

[0.3286023437976837, 0.871720016002655]


In [37]:
model.save("myFirstSaving.h5")

In [38]:
loaded_model = keras.models.load_model("myFirstSaving.h5")

In [39]:
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
_________________________________________________________________
